In [1]:
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
from transformers import AutoTokenizer
from transformers import MarianMTModel, MarianTokenizer
from transformers import T5Tokenizer, MT5ForConditionalGeneration
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

2025-06-17 17:58:34.108698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750183114.341077      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750183114.408186      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Set seed.
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [3]:
# SRC_LANGUAGE = 'ko'
# TGT_LANGUAGE = 'en'
# src_model_checkpoint = 'distilbert-base-cased'
# tgt_model_checkpoint = "klue/bert-base"

# model_name = "Helsinki-NLP/opus-mt-ko-en"

# model_name = "google/mt5-small"
model_name = "facebook/mbart-large-50-many-to-many-mmt"

training_file_path = "/kaggle/input/sampledataset/sampleData.json"
cols = ["ko_text", "en_text"]

test_size = 0.25

BATCH_SIZE = 5
NUM_EPOCHS = 1
LEARNING_RATE = 1e-4
if(torch.cuda.is_available()):
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Using {DEVICE} for processing")

Using cuda for processing


In [4]:
# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
src_text = ["Convert the following korean text to English. Input: 안녕하세요. 만나서 반갑습니다."]
inputs = tokenizer(src_text, return_tensors="pt")

inputs

NameError: name 'tokenizer' is not defined

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained(model_name)#.to(DEVICE)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
src_text = ["안녕하세요. 만나서 반갑습니다."]
src_text = ["오늘 날씨가 정말 좋네요."]
tokenizer.src_lang = "ko_KR"
inputs = tokenizer(src_text, return_tensors="pt", padding=True).to(DEVICE)

outputs = model.generate(**inputs)

translation = tokenizer.decode(outputs[0], skip_special_tokens = True)

print(f"output = {translation}")

output = It's really good weather today.


In [24]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [7]:
token_transformer = MBart50TokenizerFast.from_pretrained(model_name)
token_transformer.src_lang = "ko_KR"

In [8]:
token_transformer

MBart50TokenizerFast(name_or_path='facebook/mbart-large-50-many-to-many-mmt', vocab_size=250054, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False

In [9]:
# json_data = json.load(training_file_path)
with open(training_file_path, encoding = 'utf-8') as f:
    json_data = json.load(f)
# json_data
texts = [{col: item[col] for col in cols} for item in json_data["Text"]]

df = pd.DataFrame(texts)
df.head()

,ko_text,en_text
0,아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 ...,What needs to be done is to scoop up the boile...
1,매번 잘 차려진 밥상에 숟가락만 들었던 내 모습이 부끄러워진다.,I'm ashamed of myself for holding only a spoon...
2,가끔 평일에 일찍 들어가 가족들과 밥을 먹을 때 왜 아내가 그렇게 아이들에게 고함을...,Sometimes when I got home early on weekdays an...
3,"분명 아이들 밥을 차린 시간은 5분이 채 되지 않았는데, 아침식사 시간은 한 시간이...",It has been less than 5 minutes since the chil...


In [10]:
from datasets import load_dataset

ds = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

dev.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/448k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/166215 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1958 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1982 [00:00<?, ? examples/s]

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})

In [ ]:
# train_data, test_data = train_test_split(df, test_size = test_size)

In [12]:
# Custom Dataset class.
class TranslationDataset(Dataset):
    def __init__(self, ds, tokenizer, max_length = 128):
        self.ds = ds
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        src = self.ds['korean'][idx]
        tgt = self.ds['english'][idx]

        src_enc = self.tokenizer(src, return_tensors = "pt", padding="max_length", truncation=True, max_length = self.max_length)
        tgt_enc = self.tokenizer(tgt, return_tensors = "pt", padding="max_length", truncation=True, max_length = self.max_length)

        input_ids = src_enc["input_ids"].squeeze()
        attention_mask = src_enc["attention_mask"].squeeze()
        labels = tgt_enc["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100 #ignore padding in loss calculation
        return{
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels":labels
        }

In [13]:
train_dataset = TranslationDataset(ds['train'], token_transformer)

In [14]:
len(train_dataset)

166215

In [15]:
train_dataset = TranslationDataset(ds['train'], token_transformer)
valid_dataset = TranslationDataset(ds['validation'], token_transformer)
iterator = iter(train_dataset)
print(next(iterator))

{'input_ids': tensor([250014,     15,  19913,   2280,     16,   1504,  39048,    697,  19893,
         88986,      6,  41040,      4,  66244,   6765,  21713,     31,   5826,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,   

In [ ]:
# model = MarianMTModel.from_pretrained(model_name).to(DEVICE)

In [16]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")
print(model)

610,879,488 total parameters.
610,879,488 training parameters.
MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.98), eps=1e-9)

In [18]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [19]:
len(train_dataloader)

33243

In [20]:
for batch in tqdm(train_dataloader):
    print(batch['input_ids'].shape)
    break

  0%|          | 0/33243 [00:00<?, ?it/s]

torch.Size([5, 128])


In [21]:
train_loss = []
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
def train(model, optimizer, num_epochs, dataloader):
    print('Training')
    model.to(DEVICE)
    model.train()

    # train_loss = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        num_iter = 0
        for batch in tqdm(dataloader):
            input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
            attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
            labels = torch.tensor(batch['labels']).to(DEVICE)
    
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            epoch_loss += loss.item()
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            if((num_iter+1) % 4000 == 0):
                #save the model
                print(f"saving file")
                model.save_pretrained(f"/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-{num_iter+1}")

                #save tokenizer
                # tokenizer.save_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token")
            num_iter += 1
            print(f"Epoch {epoch}, iter = {num_iter}, Loss: {loss.item()}")
        train_loss.append(epoch_loss/num_iter)
    return train_loss

def evaluate(model, dataloader):
    print('Validating')
    model.eval()
    losses = 0
    num_iter = 0
    # valid_loss = []
    for batch in dataloader:
        input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
        attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
        labels = torch.tensor(batch['labels']).to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        losses += loss.item()
        num_iter += 1
        print(f"iter = {num_iter}, Loss: {loss.item()}")
        
    # valid_loss.append(epoch_loss/num_iter)
    return losses/num_iter

In [22]:
loss = train(model, optimizer, NUM_EPOCHS, train_dataloader)

Training


  0%|          | 0/33243 [00:00<?, ?it/s]

/tmp/ipykernel_35/2427182959.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


Epoch 0, iter = 1, Loss: 2.345632553100586
Epoch 0, iter = 2, Loss: 2.12126088142395
Epoch 0, iter = 3, Loss: 1.76872718334198
Epoch 0, iter = 4, Loss: 1.9246488809585571
Epoch 0, iter = 5, Loss: 1.9785051345825195
Epoch 0, iter = 6, Loss: 2.2000577449798584
Epoch 0, iter = 7, Loss: 2.4261202812194824
Epoch 0, iter = 8, Loss: 2.1589763164520264
Epoch 0, iter = 9, Loss: 2.164755344390869
Epoch 0, iter = 10, Loss: 2.1576361656188965
Epoch 0, iter = 11, Loss: 1.8536262512207031
Epoch 0, iter = 12, Loss: 1.809577226638794
Epoch 0, iter = 13, Loss: 1.5268950462341309
Epoch 0, iter = 14, Loss: 1.369931936264038
Epoch 0, iter = 15, Loss: 2.251371383666992
Epoch 0, iter = 16, Loss: 2.2605535984039307
Epoch 0, iter = 17, Loss: 1.2161473035812378
Epoch 0, iter = 18, Loss: 1.4974342584609985
Epoch 0, iter = 19, Loss: 1.7805230617523193
Epoch 0, iter = 20, Loss: 1.846947193145752
Epoch 0, iter = 21, Loss: 1.597293496131897
Epoch 0, iter = 22, Loss: 1.7136849164962769
Epoch 0, iter = 23, Loss: 2.18

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


...

/tmp/ipykernel_35/2427182959.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


Epoch 0, iter = 4001, Loss: 2.258314847946167
Epoch 0, iter = 4002, Loss: 2.2975943088531494
Epoch 0, iter = 4003, Loss: 2.229353904724121
Epoch 0, iter = 4004, Loss: 2.1503145694732666
Epoch 0, iter = 4005, Loss: 1.2748163938522339
Epoch 0, iter = 4006, Loss: 1.7318681478500366
Epoch 0, iter = 4007, Loss: 2.214449644088745
Epoch 0, iter = 4008, Loss: 2.070574998855591
Epoch 0, iter = 4009, Loss: 1.8142117261886597
Epoch 0, iter = 4010, Loss: 2.6696054935455322
Epoch 0, iter = 4011, Loss: 2.8003692626953125
Epoch 0, iter = 4012, Loss: 2.4377894401550293
Epoch 0, iter = 4013, Loss: 2.2360942363739014
Epoch 0, iter = 4014, Loss: 2.4961416721343994
Epoch 0, iter = 4015, Loss: 1.9365367889404297
Epoch 0, iter = 4016, Loss: 1.2151520252227783
Epoch 0, iter = 4017, Loss: 2.1576366424560547
Epoch 0, iter = 4018, Loss: 2.582512617111206
Epoch 0, iter = 4019, Loss: 1.9042949676513672
Epoch 0, iter = 4020, Loss: 1.8877217769622803
Epoch 0, iter = 4021, Loss: 1.8809696435928345
Epoch 0, iter = 40

/tmp/ipykernel_35/2427182959.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


Epoch 0, iter = 8001, Loss: 2.99222469329834
Epoch 0, iter = 8002, Loss: 2.0461268424987793
Epoch 0, iter = 8003, Loss: 1.6261430978775024
Epoch 0, iter = 8004, Loss: 2.4462122917175293
Epoch 0, iter = 8005, Loss: 3.1622066497802734
Epoch 0, iter = 8006, Loss: 2.6730518341064453
Epoch 0, iter = 8007, Loss: 2.5402204990386963
Epoch 0, iter = 8008, Loss: 2.495793342590332
Epoch 0, iter = 8009, Loss: 3.24519944190979
Epoch 0, iter = 8010, Loss: 2.032900094985962
Epoch 0, iter = 8011, Loss: 2.2664296627044678
Epoch 0, iter = 8012, Loss: 2.4366226196289062
Epoch 0, iter = 8013, Loss: 1.8083727359771729
Epoch 0, iter = 8014, Loss: 2.8253750801086426
Epoch 0, iter = 8015, Loss: 2.4255900382995605
Epoch 0, iter = 8016, Loss: 2.121415138244629
Epoch 0, iter = 8017, Loss: 2.6104555130004883
Epoch 0, iter = 8018, Loss: 2.4170925617218018
Epoch 0, iter = 8019, Loss: 2.8894600868225098
Epoch 0, iter = 8020, Loss: 2.122023582458496
Epoch 0, iter = 8021, Loss: 2.626593828201294
Epoch 0, iter = 8022, 

KeyboardInterrupt: 

In [23]:
num_iter

NameError: name 'num_iter' is not defined

In [24]:
model.save_pretrained(f"/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-{10648}")

In [26]:
from IPython.display import FileLinks
FileLinks(r'/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-4000')

/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-4000/
  model.safetensors
  config.json
  generation_config.json

In [27]:
from IPython.display import FileLinks
FileLinks(r'/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-8000')

/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-8000/
  model.safetensors
  config.json
  generation_config.json

In [28]:
from IPython.display import FileLinks
FileLinks(r'/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-10648')

/kaggle/working/mbart-large-50-many-to-many-mmt-finetuning-10648/
  model.safetensors
  config.json
  generation_config.json

In [32]:
import matplotlib.pyplot as plt

len(train_loss)

0

In [30]:
evaluate(model, valid_dataloader)

Validating


/tmp/ipykernel_35/2427182959.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/2427182959.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


iter = 1, Loss: 1.986567497253418
iter = 2, Loss: 1.7963148355484009
iter = 3, Loss: 2.144836902618408
iter = 4, Loss: 1.918829083442688
iter = 5, Loss: 2.2362546920776367
iter = 6, Loss: 1.941628098487854
iter = 7, Loss: 2.7745883464813232
iter = 8, Loss: 2.4040966033935547
iter = 9, Loss: 2.566226005554199
iter = 10, Loss: 2.063109874725342
iter = 11, Loss: 2.0746512413024902
iter = 12, Loss: 2.4752111434936523
iter = 13, Loss: 2.6076040267944336
iter = 14, Loss: 2.851728916168213
iter = 15, Loss: 3.092665910720825
iter = 16, Loss: 2.769496440887451
iter = 17, Loss: 3.139636993408203
iter = 18, Loss: 2.1632533073425293
iter = 19, Loss: 2.51161527633667
iter = 20, Loss: 2.6942286491394043
iter = 21, Loss: 2.85949969291687
iter = 22, Loss: 1.9889450073242188
iter = 23, Loss: 1.636935830116272
iter = 24, Loss: 1.647405743598938
iter = 25, Loss: 1.8891956806182861
iter = 26, Loss: 1.6774611473083496
iter = 27, Loss: 2.1117453575134277
iter = 28, Loss: 1.6889218091964722
iter = 29, Loss: 


KeyboardInterrupt



In [ ]:
#save the model
model.save_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning")

#save tokenizer
tokenizer.save_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token")

In [ ]:
#load the model
finetuned_model = MarianMTModel.from_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning").to(DEVICE)


In [ ]:
src_text = ["안녕하세요. 만나서 반갑습니다."]
inputs = tokenizer(src_text, return_tensors="pt", padding=True).to(DEVICE)

outputs = finetuned_model.generate(**inputs)

translation = tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
translation

# Prompting with mbart model

In [ ]:
#bleu score
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

src_text = "오늘 날씨가 정말 좋네요."
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
src_text = (
    "Context: Talking about the weather.\n"
    "Korean: 오늘 날씨가 정말 좋네요.\n"
    "English:"
)

src_text1 = (
    "Context: A Teacher is discussing about a good scoring student with his parents\n"
    "Korean: 재능만 중요한 게 아닙니다. 학생이 얼마나 노력했는지도 봐야 합니다.\n"
    "English:"
)

src_text2 = "재능만 중요한 게 아닙니다. 학생이 얼마나 노력했는지도 봐야 합니다."
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text1, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
src_text2 = "재능만 중요한 게 아닙니다. 학생이 얼마나 노력했는지도 봐야 합니다."
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text2, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
src_text = (
    "Context: output the translation in json format.\n"
    "Mr. Kim: 오늘 날씨가 정말 좋네요.\n"
    "English:"
)
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
src_text = (
    "Context: Complete the sentence given below.\n"
    "English: It was all good until"
)
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer.src_lang = "en_XX"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# src_text = "Hello world, here I am"
src_text = (
    "Context: Provide output in json format \n"
    "English: It was all good until\n"
    "Korean:"
)
tokenizer.src_lang = "en_XX"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [ ]:
from datasets import load_dataset

ds = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")

In [ ]:
weights = (0.25, 0.25, 0, 0) 
for i in range(len(ds['test']['korean'])):
    src_text = ds['test']['korean'][i]
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    print(predictions)
    reference = ds['test']['english'][i]
    print(reference)
    # print(ds['test']['english'][i])
    score = sentence_bleu(reference, predictions, weights = weights)
    print(score)
    break

In [ ]:
ds['test']['english'][0]

In [ ]:
# pip install torchao
import torch
from transformers import TorchAoConfig, AutoModelForCausalLM, AutoTokenizer

quantization_config = TorchAoConfig("int4_weight_only", group_size=128)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-13b-hf",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
input_ids = tokenizer("Plants create energy through a process known as", return_tensors="pt").to("cuda")

output = model.generate(**input_ids, cache_implementation="static")
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# pip install torchao
import torch
from transformers import TorchAoConfig, AutoModelForCausalLM, AutoTokenizer

quantization_config = TorchAoConfig("int4_weight_only", group_size=128)
model = AutoModelForCausalLM.from_pretrained(
    "huggyllama/llama-7b",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-13b")
input_ids = tokenizer("Plants create energy through a process known as", return_tensors="pt").to("cuda")

output = model.generate(**input_ids, cache_implementation="static")
print(tokenizer.decode(output[0], skip_special_tokens=True))